In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from telegram import Update, ReplyKeyboardMarkup
from telegram.ext import Application, CommandHandler, MessageHandler, CallbackContext, filters
import pandas as pd
import os

TOKEN = "7557717542:AAG01tYbv_MqGF6QfYlpUAdE_sAlp-Kzdpk"

# فایل ذخیره نوبت‌ها
FILE_NAME = "appointments.csv"

# ایجاد فایل CSV در صورت نبودن
if not os.path.exists(FILE_NAME):
    df = pd.DataFrame(columns=["Name", "Phone", "Date", "Time"])
    df.to_csv(FILE_NAME, index=False)

# 📌 تابع شروع ربات
async def start(update: Update, context: CallbackContext):
    await update.message.reply_text("سلام! 😊 به ربات نوبت‌دهی خوش اومدی.\n"
                                    "برای گرفتن نوبت، دستور /reserve رو بزن.\n"
                                    "برای دیدن لیست نوبت‌ها، دستور /appointments رو بزن.\n"
                                    "برای کنسل کردن نوبت، دستور /cancel رو بزن.")

# 📌 دریافت نام و ذخیره‌سازی آن
async def get_name(update: Update, context: CallbackContext):
    context.user_data["name"] = update.message.text
    await update.message.reply_text("شماره تلفن خود را وارد کنید:")
    context.user_data["next_step"] = "get_phone"

# 📌 دریافت شماره تلفن و ذخیره نوبت
async def get_phone(update: Update, context: CallbackContext):
    phone = update.message.text
    name = context.user_data.get("name")

    context.user_data["phone"] = phone
    await update.message.reply_text("تاریخ نوبت خود را وارد کنید (مثلاً: 2025-03-10):")
    context.user_data["next_step"] = "get_date"

# 📌 دریافت تاریخ نوبت
async def get_date(update: Update, context: CallbackContext):
    date = update.message.text
    context.user_data["date"] = date

    # ایجاد لیست ساعات در دسترس
    available_times = [f"{hour:02}:00" for hour in range(9, 18, 1)] + \
                      [f"{hour:02}:30" for hour in range(9, 17, 1)]

    # خواندن نوبت‌های ثبت‌شده برای بررسی زمان‌های اشغال‌شده
    df = pd.read_csv(FILE_NAME)
    booked_times = df[df["Date"] == date]["Time"].tolist()
    free_times = [t for t in available_times if t not in booked_times]

    if not free_times:
        await update.message.reply_text("متأسفم! 😞 تمام نوبت‌های این روز پر شده‌اند. لطفاً تاریخ دیگری را انتخاب کنید.")
        return

    # ارسال لیست زمان‌های آزاد به کاربر
    keyboard = [[t] for t in free_times]
    reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    await update.message.reply_text("لطفاً یکی از ساعت‌های زیر را انتخاب کنید:", reply_markup=reply_markup)

    context.user_data["next_step"] = "get_time"

# 📌 دریافت ساعت نوبت و ثبت آن
async def get_time(update: Update, context: CallbackContext):
    time = update.message.text
    name = context.user_data.get("name")
    phone = context.user_data.get("phone")
    date = context.user_data.get("date")

    # ذخیره در فایل CSV
    df = pd.read_csv(FILE_NAME)
    new_appointment = pd.DataFrame([[name, phone, date, time]], columns=df.columns)
    df = pd.concat([df, new_appointment], ignore_index=True)
    df.to_csv(FILE_NAME, index=False)

    await update.message.reply_text(f"✅ نوبت شما برای {date} ساعت {time} ثبت شد!\n"
                                    "برای مشاهده لیست نوبت‌ها، دستور /appointments رو بزن.")

# 📌 نمایش لیست نوبت‌ها
async def show_appointments(update: Update, context: CallbackContext):
    df = pd.read_csv(FILE_NAME)

    if df.empty:
        await update.message.reply_text("هیچ نوبتی ثبت نشده است.")
    else:
        text = "📅 **لیست نوبت‌های ثبت‌شده:**\n\n"
        for _, row in df.iterrows():
            text += f"👤 {row['Name']} | 📞 {row['Phone']} | 📆 {row['Date']} | 🕒 {row['Time']}\n"
        
        await update.message.reply_text(text)

# 📌 کنسل کردن نوبت
async def cancel_appointment(update: Update, context: CallbackContext):
    await update.message.reply_text("لطفاً شماره تلفن خود را وارد کنید تا نوبت شما حذف شود.")
    context.user_data["next_step"] = "delete_appointment"

async def delete_appointment(update: Update, context: CallbackContext):
    phone = update.message.text
    df = pd.read_csv(FILE_NAME)

    if phone in df["Phone"].values:
        df = df[df["Phone"] != phone]
        df.to_csv(FILE_NAME, index=False)
        await update.message.reply_text("❌ نوبت شما با موفقیت حذف شد.")
    else:
        await update.message.reply_text("شماره تلفن شما در لیست نوبت‌ها پیدا نشد.")

# 📌 پردازش پیام‌های کاربران
async def handle_messages(update: Update, context: CallbackContext):
    next_step = context.user_data.get("next_step")

    if next_step == "get_name":
        await get_name(update, context)
    elif next_step == "get_phone":
        await get_phone(update, context)
    elif next_step == "get_date":
        await get_date(update, context)
    elif next_step == "get_time":
        await get_time(update, context)
    elif next_step == "delete_appointment":
        await delete_appointment(update, context)
    else:
        await update.message.reply_text("دستور نامعتبر! لطفاً از دستورات ربات استفاده کنید.")

# 📌 راه‌اندازی ربات
app = Application.builder().token(TOKEN).build()
app.add_handler(CommandHandler("start", start))
app.add_handler(CommandHandler("reserve", lambda u, c: (u.message.reply_text("نام خود را وارد کنید:"), c.user_data.update({"next_step": "get_name"}))))
app.add_handler(CommandHandler("appointments", show_appointments))
app.add_handler(CommandHandler("cancel", cancel_appointment))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_messages))

# اجرا
app.run_polling()


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\httpx\_transports\default.py", line 69, in map_httpcore_exceptions
    yield
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\httpx\_transports\default.py", line 373, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\httpcore\_async\connection_pool.py", line 268, in handle_async_request
    raise exc
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\httpcore\_async\connection_pool.py", line 251, in handle_async_request
    response = await connection.handle_async_request(request)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\httpcore\_async\http_proxy.py", line 344, in handle_async_request
    return await self

In [ ]:
import httpx

client = httpx.AsyncClient(timeout=httpx.Timeout(10.0))  # timeout را افزایش دهید
